## Generazione di testate giornalistiche con l'utilizzo di reti neurali
Le reti neurali artificiali sono sistemi che permettono ai computer di riconoscere pattern e risolvere problemi nello strato di output passando per gli strati intermedi. Per l'aggiornamento dei pesi durante il train viene utilizzato il calcolo del gradiente, come previsto dall'algoritmo gradient descent.
Un' architettura molto diffusa è la Recurrent Neural Network. Nelle RNN sono previsti dei collegamenti tra neuroni che generano un loop. In questo modo è possibile utilizzare input sequenziali, come valori raccolti nel tempo, per effettuare predizioni sui valori futuri. A seguito della presentazione dei dati di input la rete produrrà una coppia di valori: uno verrà restitutito come output mentre l'altro sarà a sua volta ri-passato come input. Se il modello è addesrato correttamente allora avrà "dedotto" la sequenza che lega i dati ed effettuerà predizioni consistenti e significative.

Le RNN hanno un problema però: man mano che vengono effettuate predizioni, i dati di input (che devono guidare la produzione dell'output del stema) saranno sempre meno "presenti". Dunque più ci si "alllontana" dall'input e peggiore sarà la qualità del risulato. Allo stesso modo il training non può essere effettuato come accadeva con le reti feed forward. Infatti l'aggioramento dei pesi secondo l'algoritmo gradient decsent prevede che maggiore è il gradiente, maggiore è il cambiamento che viene effettuato sui pesi di un certo strato. Analogamente se ad un certo strato saranno stati applicate piccole  modifiche, i cambiamenti allo strato precedente saranno ancora minori e dunque il gradiente si abbasserà esponenzialmente man mano che avviene la retropropagazione. Questo è il problema del gradient vanishing. È questo il motivo per il quale nelle reti ricorrenti si ha questo degradamento della qualità della predizione in funzione della lunghezza dell'output.
Per risolvere questo problema si introdcono delle architetture più complesse dotate di unità interne chiamate GATES che contengono operazioni per permettere selezionare quali informazioni mantenere e quali eliminare ad ogni loop. Questi modelli sono le Long Short-Term memory e le GRU. Questa capacità delle reti ricorrenti di mantenere in qualche modo parte dell'informazione delle predizioni precedenti ha conferito alle RNN la nomea di reti con "memoria".

#### Le RNN possono essere utilizzare per generare testo
La generazione è fatta attraverso ua predizione del un termine uccessivo rispetto ad una lista di termini. I termini vanno dunqe tokenizzati e la rete si addestra sulla probabilità di leggere un certo termine successivamente ad un'altro (o più).

Vediamo ora un esempio di applicazione di una LSTM per la generazione di testate giornalistiche.
Il dati utilizzati sono presi dal dataset pubblico "News Category Dataset" (v3).

### Per prima cosa sono importate tutte le librerie necessarie

In [1]:
    # keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 

    # set seeds for reproducability
#from tensorflow import set_random_seed <---- non c'è bisogno in tensorflow2
import tensorflow
from numpy.random import seed
#set_random_seed(2) <----- questo è sempre per la vecchia versione di tf
tensorflow.random.set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import json
import csv
import os

import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

### Poi si carica il dataset e si conservano solo le informazioni utili a questo task

In [2]:
df = pd.read_json('News_Category_Dataset_v3.json', lines=True)
df.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [3]:
all_headlines = df['headline']
len(all_headlines)

209527

In [4]:
all_headlines.head()

0    Over 4 Million Americans Roll Up Sleeves For O...
1    American Airlines Flyer Charged, Banned For Li...
2    23 Of The Funniest Tweets About Cats And Dogs ...
3    The Funniest Tweets From Parents This Week (Se...
4    Woman Who Called Cops On Black Bird-Watcher Lo...
Name: headline, dtype: object

In [101]:
all_headlines = df['headline']
all_headlines = all_headlines[:8000]       #se carico tutti i 209527 mi va in errore di out of memory. Con 25'000 record funziona ma ci mette tantissimo ad addestrare quindi ho tagliato fino a 8'000 
all_headlines.head()

0    Over 4 Million Americans Roll Up Sleeves For O...
1    American Airlines Flyer Charged, Banned For Li...
2    23 Of The Funniest Tweets About Cats And Dogs ...
3    The Funniest Tweets From Parents This Week (Se...
4    Woman Who Called Cops On Black Bird-Watcher Lo...
Name: headline, dtype: object

### Passiamo ora alla preparazione del dataset
#### Per prima cosa puliamo il dataset

In [102]:
def clean_text(txt):
    txt = "".join(v for v in txt if v not in string.punctuation).lower()
    txt = txt.encode("utf8").decode("ascii",'ignore')
    return txt 

corpus = [clean_text(x) for x in all_headlines]
corpus[:10]

['over 4 million americans roll up sleeves for omicrontargeted covid boosters',
 'american airlines flyer charged banned for life after punching flight attendant on video',
 '23 of the funniest tweets about cats and dogs this week sept 1723',
 'the funniest tweets from parents this week sept 1723',
 'woman who called cops on black birdwatcher loses lawsuit against exemployer',
 'cleaner was dead in belk bathroom for 4 days before body found police',
 'reporter gets adorable surprise from her boyfriend while live on tv',
 'puerto ricans desperate for water after hurricane fionas rampage',
 'how a new documentary captures the complexity of being a child of immigrants',
 'biden at un to call russian war an affront to bodys charter']

#### Poi generiamo sequene di n-grammi come token

In [103]:
tokenizer = Tokenizer()

def get_sequence_of_tokens(corpus):
    ## tokenization
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    
    ## convert data to sequence of tokens 
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    return input_sequences, total_words

inp_sequences, total_words = get_sequence_of_tokens(corpus)
inp_sequences[:10]

[[19, 151],
 [19, 151, 87],
 [19, 151, 87, 257],
 [19, 151, 87, 257, 1943],
 [19, 151, 87, 257, 1943, 37],
 [19, 151, 87, 257, 1943, 37, 7087],
 [19, 151, 87, 257, 1943, 37, 7087, 5],
 [19, 151, 87, 257, 1943, 37, 7087, 5, 7088],
 [19, 151, 87, 257, 1943, 37, 7087, 5, 7088, 163],
 [19, 151, 87, 257, 1943, 37, 7087, 5, 7088, 163, 4926]]

### Ed infine aggiungiamo padding alle sequenze per ottenere le variabili predictors e target

(esempio: per le variabili predictors ["bevo", "un", "bicchiere", "di"] si associa una variabile target ["acqua"]).

In [104]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

### Creiamo ora il modello LSTM...

In [105]:
def create_model(max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    
    # Add Input Embedding Layer
    model.add(Embedding(total_words, 10, input_length=input_len))
    
    # Add Hidden Layer 1 - LSTM Layer
    model.add(LSTM(100))
    model.add(Dropout(0.1))
    
    # Add Output Layer
    model.add(Dense(total_words, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

model = create_model(max_sequence_len, total_words)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 18, 10)            140350    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 14035)             1417535   
Total params: 1,602,285
Trainable params: 1,602,285
Non-trainable params: 0
_________________________________________________________________


### ...E poi addestriamolo su 100 epoche.

In [106]:
#Lets train our model now
model.fit(predictors, label, epochs=100, verbose=5)

Epoch 1/100
Epoch 2/100
Epoch 3/100
Epoch 4/100
Epoch 5/100
Epoch 6/100
Epoch 7/100
Epoch 8/100
Epoch 9/100
Epoch 10/100
Epoch 11/100
Epoch 12/100
Epoch 13/100
Epoch 14/100
Epoch 15/100
Epoch 16/100
Epoch 17/100
Epoch 18/100
Epoch 19/100
Epoch 20/100
Epoch 21/100
Epoch 22/100
Epoch 23/100
Epoch 24/100
Epoch 25/100
Epoch 26/100
Epoch 27/100
Epoch 28/100
Epoch 29/100
Epoch 30/100
Epoch 31/100
Epoch 32/100
Epoch 33/100
Epoch 34/100
Epoch 35/100
Epoch 36/100
Epoch 37/100
Epoch 38/100
Epoch 39/100
Epoch 40/100
Epoch 41/100
Epoch 42/100
Epoch 43/100
Epoch 44/100
Epoch 45/100
Epoch 46/100
Epoch 47/100
Epoch 48/100
Epoch 49/100
Epoch 50/100
Epoch 51/100
Epoch 52/100
Epoch 53/100
Epoch 54/100
Epoch 55/100
Epoch 56/100
Epoch 57/100
Epoch 58/100
Epoch 59/100
Epoch 60/100
Epoch 61/100
Epoch 62/100
Epoch 63/100
Epoch 64/100
Epoch 65/100
Epoch 66/100
Epoch 67/100
Epoch 68/100
Epoch 69/100
Epoch 70/100
Epoch 71/100
Epoch 72/100
Epoch 73/100
Epoch 74/100
Epoch 75/100
Epoch 76/100
Epoch 77/100
Epoch 78

### Ora effettuiamo delle prove di generazione

Per generare il testo è necessario specificare l'inizio del testo e la lunghezza dell'output

In [107]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ""
        for word,index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " "+output_word
    return seed_text.title()

In [108]:
print (generate_text("united states", 5, model, max_sequence_len))

United States Launch Series Of Covid19 Cases


In [113]:
print (generate_text("preident trump", 8, model, max_sequence_len))

Preident Trump Says He Will No Longer Abide With Coronavirus


In [117]:
print (generate_text("donald trump", 5, model, max_sequence_len))

Donald Trump Is A Baby On Board


In [120]:
print (generate_text("india and china", 4, model, max_sequence_len))

India And China Who Pulled From Allegedly


In [121]:
print (generate_text("new york", 4, model, max_sequence_len))

New York State Announces Abortion Ban


In [122]:
print (generate_text("science and technology", 5, model, max_sequence_len))

Science And Technology To Take Your Western Security


In [123]:
print (generate_text("Queen", 12, model, max_sequence_len))

Queen Elizabeth Gracefully Declines Oldie Award With Brilliant Response To Space West In


In [124]:
print (generate_text("Coffee", 16, model, max_sequence_len))

Coffee Condemn The Most Calling Fight Should Be Like On The World Democrat Moment Is A Bear


### Considerazioni
Nonostante i risultati ottenuti siano in generale poco credibili, mostrano comuqnue che As we can see, the model has produced the output which looks fairly fine. The results can be improved further with following points:

- Aggiungere dati
- Effettuare Fine Tuning tdell'architettura della rete
- Effettuare Fine Tuning dei parametri della rete